In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import importlib

#for sliders this will install ipywidgets
try:
    importlib.import_module("ipywidgets")
except ImportError:
    import pip
    pip.main(['install', 'ipywidgets'])
    
    
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [29]:
def plot_solution(x0,days,alpha,beta,N,k0,k,gamma,sigma,f,mu,lkdwn_start,lkdwn_end,k0_lkdwn,k_lkdwn):
    
    gamma=gamma/100 #gamma is between 1/14 to 1/7 so, 1/100 is to scale the value assigned by the slider
    
    def sirx(x,t):
        #import the values
        s,i,xl,xq,e,r=x
        '''
        #defining the parameters
        if t>lkdwn_start and t<lkdwn_end:
            k0=k0_lkdwn
            k=k_lkdwn
        else:
            k0=k0
            k=k
        '''
        lam=(i+e)*beta/N
        lkdwn_start1=list(map(int,lkdwn_start.split(',')))
        lkdwn_end1=list(map(int,lkdwn_end.split(',')))
        
        for j in range(len(lkdwn_start1)):
            if t>lkdwn_start1[j] and t<lkdwn_end1[j]:
                lock=1
                break
            else:
                lock=0
        
        #THE EQUATIONS
        if lock==1:
            dsdt = -(lam+k0_lkdwn)*s + (mu*xl)
            didt = alpha*lam*s - (gamma+k_lkdwn)*i + sigma*e
            dxldt = k0_lkdwn*s - (f*lam + mu)*xl + k0_lkdwn*e
            dxqdt = k_lkdwn*i + (f*lam*xl) - (mu*xq)
            dedt = ((1-alpha)*lam*s) - ((gamma+sigma+k0_lkdwn)*e)
            drdt = gamma*(i + e) + (mu*xq)
        else:
            dsdt = -(lam+k0)*s + (mu*xl)
            didt = alpha*lam*s - (gamma+k)*i + sigma*e
            dxldt = k0*s - (f*lam + mu)*xl + k0*e
            dxqdt = k*i + (f*lam*xl) - (mu*xq)
            dedt = ((1-alpha)*lam*s) - ((gamma+sigma+k0)*e)
            drdt = gamma*(i + e) + (mu*xq)

        return [dsdt,didt,dxldt,dxqdt,dedt,drdt]
    
    #Ploting the equations
    t=np.linspace(0,days,(days*1000))
    x=odeint(sirx,x0,t)
    s = x[:,0]
    i = x[:,1]
    xl = x[:,2]
    xq = x[:,3]
    e = x[:,4]
    r = x[:,5]
    arr=['Suceptible','Infected','Lockdown','Quarantined','Exposed','Removed']
    for i in range(6):
        plt.subplot(2,3,i+1)
        plt.subplots_adjust(hspace =1,wspace=1)
        plt.plot(t,(x[:,i]/N)*100)
        plt.ylabel(arr[i])
        plt.xlabel("Time in Days")
    plt.show()
    plt.plot(t,(x/N)*100)
    plt.legend(arr)
    text="days={} alpha={} beta={} population={} \n k0={} k={} gamma={} sigma={} f={} mu={} \n lkdwn_start={} lkdwn_end={} k0_lkdwn={} k_lkdwn={}".format(days,alpha,beta,N,k0,k,gamma,sigma,f,mu,lkdwn_start,lkdwn_end,k0_lkdwn,k_lkdwn)
    plt.xlabel("Time in Day"+"\n"+text)
    plt.show()

In [31]:
days=200 #No. of days of simulation

#initial valus of the [suceptible, infected, lockdown, quarantined, asymptomatic_carrier, removed] people sequence is fixed
n=1000000           # Total population
n_i=1              # Infected people
n_xl=0             # Lockdown
n_xq=0             # Quarantined
n_e=0              # Asymptomatic 
n_r=0              # Removed people
n_s= n - n_i - n_e # Suceptible people
print(n_s)
x0=[n_s,n_i,n_xl,n_xq,n_e,n_r]

w = interactive(plot_solution, x0=fixed(x0), days=fixed(days),\
                #chage the values in brackets to change the range
                alpha=(0.0,1.0),\
                beta=(0.0,10.0),N=fixed(n),\
                k0=(0.0,5.0),\
                k=(0.0,10.0),\
                gamma=(1.0,14.2),\
                sigma=(0.0,1.0),\
                f=(0.0,1.0),\
                mu=(0.0,1.0),\
                lkdwn_start=input('Lockdown start Dates'),\
                lkdwn_end=input('Lockdown end Dates'),\
                k0_lkdwn=(0.0,5.0),\
                k_lkdwn=(0.0,10.0))
display(w)

999999
Lockdown start Dates30,70
Lockdown end Dates51,100


interactive(children=(FloatSlider(value=0.5, description='alpha', max=1.0), FloatSlider(value=5.0, description…